In [1]:
import urllib.request
import urllib.parse
import json
import datetime
import plotly.graph_objs as go
import sqltables.sqlite3

In [2]:
db = sqltables.sqlite3.Database()

In [2]:
url = "https://webservice-eex.gvsi.com/query/json/getHistoricalTick/offtradeprice/ontradeprice/offexchsingletradevolume/onexchsingletradevolume/tradedatetimegmt/?priceSymbol=%22%2FE.F719K22%22&maxrowcount=1000&chartstopdate=2022%2F05%2F18&chartstoptime=22%3A00%3A00"

In [16]:
urllib.parse.parse_qs("priceSymbol=%22%2FE.F719K22%22&maxrowcount=1000&chartstopdate=2022%2F05%2F18&chartstoptime=22%3A00%3A00")

{'priceSymbol': ['"/E.F719K22"'],
 'maxrowcount': ['1000'],
 'chartstopdate': ['2022/05/18'],
 'chartstoptime': ['22:00:00']}

In [33]:

urllib.parse.parse_qs("optionroot=%22%2FE.DB_DAILY%22&expirationdate=2022%2F05%2F11&onDate=2022%2F05%2F12")

{'optionroot': ['"/E.DB_DAILY"'],
 'expirationdate': ['2022/05/11'],
 'onDate': ['2022/05/12']}

In [56]:
base_url = "https://webservice-eex.gvsi.com/query/json/getHistoricalTick/offtradeprice/ontradeprice/offexchsingletradevolume/onexchsingletradevolume/tradedatetimegmt/"

In [60]:
parameters = {
#     "priceSymbol": '"/E.F719K22"',
    "priceSymbol": '"/E.DB16X21"',
    "maxrowcount": "1000",
    "chartstopdate": "2021/11/17",
    "chartstoptime": "22:00:00"
}


In [62]:
def get_symbols(optionroot):
base_url = "https://webservice-eex.gvsi.com/query/json/getChain/gv.pricesymbol/gv.displaydate/gv.expirationdate/tradedatetimegmt/gv.eexdeliverystart/ontradeprice/close/onexchsingletradevolume/onexchtradevolumeeex/offexchtradevolumeeex/openinterest/"


In [112]:
parameters = {
    'optionroot': ['"/E.DB_DAILY"'],
    'expirationdate': ['2021/11/11'],
    'onDate': ['2022/05/29']
}

In [5]:
def get_eex(base_url, parameters):
    url = base_url + "?" + urllib.parse.urlencode(parameters, doseq=True)
    request = urllib.request.Request(url, headers={"Origin": "https://www.eex.com", "Referer": "https://www.eex.com/"})

    with urllib.request.urlopen(request) as response:
        data = json.load(response)
    return data

In [6]:
def get_symbols(optionroot, date):
    base_url = "https://webservice-eex.gvsi.com/query/json/getChain/gv.pricesymbol/gv.expirationdate/gv.eexdeliverystart"
    parameters = {
        'optionroot': f'"{optionroot}"',
        'onDate': date
    }
    response = get_eex(base_url, parameters)
    return response["results"]["items"]

symbols = get_symbols("/E.DEB_WEEK", "2022/05/02")
symbols

[{'gv.pricesymbol': '/E.DEB02K22',
  'gv.expirationdate': '5/9/2022',
  'gv.eexdeliverystart': '5/2/2022 10:00:00 PM'},
 {'gv.pricesymbol': '/E.DEB03K22',
  'gv.expirationdate': '5/16/2022',
  'gv.eexdeliverystart': '5/9/2022 10:00:00 PM'},
 {'gv.pricesymbol': '/E.DEB04K22',
  'gv.expirationdate': '5/23/2022',
  'gv.eexdeliverystart': '5/16/2022 10:00:00 PM'},
 {'gv.pricesymbol': '/E.DEB05K22',
  'gv.expirationdate': '5/30/2022',
  'gv.eexdeliverystart': '5/23/2022 10:00:00 PM'},
 {'gv.pricesymbol': '/E.DEB01M22',
  'gv.expirationdate': '6/6/2022',
  'gv.eexdeliverystart': '5/30/2022 10:00:00 PM'}]

In [39]:
symbol_info = symbols[2]
symbol_info

{'gv.pricesymbol': '/E.DEB04K22',
 'gv.expirationdate': '5/23/2022',
 'gv.eexdeliverystart': '5/16/2022 10:00:00 PM'}

In [54]:
price_symbol = symbol_info["gv.pricesymbol"]
expiration_dt = datetime.datetime.strptime(symbol_info["gv.expirationdate"], "%m/%d/%Y")
price_symbol = "/E.DEBMJ22"
expiration_dt = datetime.datetime(2022, 4, 30)
expiration_date = expiration_dt.strftime("%Y/%m/%d")
parameters = {
    "priceSymbol": f'"{price_symbol}"',
    "maxrowcount": "1000",
    "chartstopdate": expiration_date,
    "chartstartdate": "2022/03/14",
#    "chartstoptime": "22:00:00"
}
# parameters = {
# #     "priceSymbol": '"/E.F719K22"',
# #     "priceSymbol": '"/E.DB16X21"',
#     "priceSymbol": f'"{price_symbol}"',    
#     "maxrowcount": "1000",
#     "chartstopdate": "2021/11/17",
#     "chartstoptime": "22:00:00"
# }



base_url = "https://webservice-eex.gvsi.com/query/json/getHistoricalTick/offtradeprice/ontradeprice/offexchsingletradevolume/onexchsingletradevolume/tradedatetimegmt/"
response = get_eex(base_url, parameters)
items = response["results"]["items"]
len(items)

391

In [55]:
parameters

{'priceSymbol': '"/E.DEBMJ22"',
 'maxrowcount': '1000',
 'chartstopdate': '2022/04/30',
 'chartstartdate': '2022/03/14'}

In [49]:
items[0]

{'offtradeprice': 201,
 'ontradeprice': None,
 'offexchsingletradevolume': 744,
 'onexchsingletradevolume': None,
 'tradedatetimegmt': '5/2/2022 6:49:56 AM'}

In [56]:
columns = {"tradedatetime": "text", "price": "float", "volume": "float", "onexchange": "int"}
price_tab = db.create_table(column_names=columns, column_types=columns)

for item in items:
    dt = datetime.datetime.strptime(item["tradedatetimegmt"], "%m/%d/%Y %H:%M:%S %p")
    if item["offtradeprice"] is not None:
        price = item["offtradeprice"]
        volume = item["offexchsingletradevolume"]
        onexchange = False
    else:
        price = item["ontradeprice"]
        volume = item["onexchsingletradevolume"]
        onexchange = True
    price_tab.insert([[dt.isoformat(), price, volume, onexchange]])

price_tab

|tradedatetime|price|volume|onexchange|
|-|-|-|-|
|\'2022\-03\-31T06\:14\:02\'|230\.0|720\.0|1|
|\'2022\-03\-31T06\:18\:15\'|230\.0|3600\.0|1|
|\'2022\-03\-31T06\:18\:59\'|230\.0|3600\.0|1|
|\'2022\-03\-31T06\:26\:48\'|226\.0|720\.0|1|
|\'2022\-03\-31T06\:29\:02\'|222\.0|720\.0|1|
|\'2022\-03\-31T06\:29\:17\'|222\.0|3600\.0|1|
|\'2022\-03\-31T06\:31\:11\'|222\.0|2880\.0|0|
|\'2022\-03\-31T06\:32\:04\'|222\.0|720\.0|1|
|\'2022\-03\-31T06\:33\:34\'|227\.0|3600\.0|0|
|\'2022\-03\-31T06\:33\:48\'|222\.0|720\.0|1|
|\'2022\-03\-31T06\:34\:46\'|None|3600\.0|1|
|\'2022\-03\-31T06\:36\:43\'|225\.0|720\.0|0|
|\'2022\-03\-31T06\:36\:58\'|221\.0|720\.0|1|
|\'2022\-03\-31T06\:37\:12\'|222\.0|3600\.0|1|
|\'2022\-03\-31T06\:37\:15\'|222\.0|3600\.0|1|
|\'2022\-03\-31T06\:37\:25\'|220\.0|720\.0|1|
|...|...|...|...|


In [57]:
dates, prices, volumes = zip(*price_tab.view("select tradedatetime, price, volume from _ order by tradedatetime"))
layout = {"yaxis2": {"side": "right", "overlaying": "y1"}, "title": f"{price_symbol} ({expiration_dt.isoformat()})"}
fig = go.Figure(data=[
    {"x": dates, "y": prices, "line": {"shape": "hv"}}, 
    {"x": dates, "y": volumes, "yaxis": "y2"}
], layout=layout)
fig.show()

In [32]:
volumes

(1680.0,
 2520.0,
 840.0,
 840.0,
 840.0,
 840.0,
 840.0,
 504.0,
 336.0,
 2520.0,
 840.0,
 4200.0,
 840.0,
 840.0,
 1344.0,
 168.0,
 1680.0,
 1680.0,
 840.0,
 1008.0,
 2520.0,
 840.0,
 168.0,
 168.0,
 168.0,
 168.0,
 168.0,
 1680.0,
 840.0,
 1344.0,
 168.0,
 840.0,
 672.0,
 840.0,
 4200.0,
 1680.0,
 840.0,
 1680.0,
 840.0,
 840.0,
 840.0,
 840.0,
 840.0,
 840.0,
 840.0,
 504.0,
 1680.0,
 336.0,
 1680.0,
 1344.0,
 1680.0,
 1680.0,
 168.0,
 840.0,
 840.0,
 840.0,
 840.0,
 840.0,
 2520.0,
 1680.0,
 4200.0,
 840.0,
 840.0,
 2688.0,
 336.0,
 840.0,
 840.0,
 336.0,
 1680.0,
 1680.0,
 168.0,
 2352.0,
 840.0,
 840.0,
 840.0,
 168.0,
 840.0,
 840.0,
 1680.0,
 840.0,
 840.0,
 840.0,
 840.0,
 1680.0,
 840.0,
 840.0,
 840.0,
 840.0,
 840.0,
 840.0,
 840.0,
 840.0,
 672.0,
 840.0,
 504.0,
 504.0,
 840.0,
 840.0,
 840.0,
 672.0,
 840.0,
 840.0,
 840.0,
 840.0,
 840.0,
 3360.0,
 840.0,
 840.0,
 840.0,
 1680.0,
 840.0,
 840.0,
 4200.0,
 840.0,
 840.0,
 3360.0,
 840.0,
 840.0,
 840.0,
 4200.0,
 1680.0

In [124]:
prices

[180,
 180,
 180,
 180,
 None,
 None,
 179,
 179,
 183,
 183,
 183,
 183,
 None,
 None,
 186.9,
 186,
 186,
 183,
 183,
 None,
 187,
 None,
 187,
 None,
 None,
 187.85,
 188.5,
 188.5,
 188.5,
 188.5,
 188.5,
 189,
 189,
 188,
 189,
 189,
 189,
 None,
 None,
 None,
 None,
 None,
 191,
 None,
 193,
 193,
 193,
 189.5,
 None,
 197,
 None,
 189,
 186,
 None,
 None,
 185.25,
 None,
 None,
 186.05,
 None,
 185.5,
 None,
 None,
 184,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 183,
 180,
 180,
 173,
 173,
 173,
 174,
 173,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 173,
 None,
 None,
 174,
 174,
 173,
 173,
 174,
 None,
 None,
 None,
 None,
 None,
 None,
 173,
 173,
 173,
 173,
 173,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 168,
 None,
 168.5,
 168.5,
 168.5,
 169,
 169,
 None,
 168.25,
 168.25,
 168,
 167,
 None,
 None,
 None,
 None,
 None,
 None,
 170,
 173,
 173,
 None,
 None,
 170.95,
 None,
 None,
 None

In [126]:
items[5]

{'offtradeprice': None,
 'ontradeprice': 179,
 'offexchsingletradevolume': None,
 'onexchsingletradevolume': 4200,
 'tradedatetimegmt': '5/3/2022 9:22:56 AM'}

In [111]:
items[0], items[-1]

({'offtradeprice': None,
  'ontradeprice': 190,
  'offexchsingletradevolume': None,
  'onexchsingletradevolume': 1680,
  'tradedatetimegmt': '11/16/2021 9:13:13 AM'},
 {'offtradeprice': None,
  'ontradeprice': 249,
  'offexchsingletradevolume': None,
  'onexchsingletradevolume': 1344,
  'tradedatetimegmt': '12/3/2021 4:28:31 PM'})

In [90]:
parameters

{'priceSymbol': '"/E.DB12X21"',
 'maxrowcount': '1000',
 'chartstopdate': '2021/12/11',
 'chartstoptime': '22:00:00'}

In [ ]:
{'gv.pricesymbol': '/E.DB16X21',
    'gv.displaydate': '11/16/2021',
    'gv.expirationdate': '11/16/2021',
    'tradedatetimegmt': '11/12/2021 12:00:00 PM',
    'gv.eexdeliverystart': '11/16/2021 11:00:00 PM',
    'ontradeprice': 226,
    'close': 226.04,
    'onexchsingletradevolume': 240,
    'onexchtradevolumeeex': 240,
    'offexchtradevolumeeex': 1800,
    'openinterest': 85}

{'gv.pricesymbol': '/E.DB16X21',
    'gv.displaydate': '11/16/2021',
    'gv.expirationdate': '11/16/2021',
    'tradedatetimegmt': '11/16/2021 12:00:00 PM',
    'gv.eexdeliverystart': '11/16/2021 11:00:00 PM',
    'ontradeprice': None,
    'close': 237.47,
    'onexchsingletradevolume': None,
    'onexchtradevolumeeex': None,
    'offexchtradevolumeeex': None,
    'openinterest': 2014}

In [38]:
url

' https://webservice-eex.gvsi.com/query/json/getChain/gv.pricesymbol/gv.displaydate/gv.expirationdate/tradedatetimegmt/gv.eexdeliverystart/ontradeprice/close/onexchsingletradevolume/onexchtradevolumeeex/offexchtradevolumeeex/openinterest/?priceSymbol=%22%2FE.F719K22%22&maxrowcount=1000&chartstopdate=2022%2F04%2F20&chartstoptime=22%3A00%3A00'

{'results': {'items': []}}